# Image-Classification-For-Cross-Analysis-of-Chest-X-Ray

In [1]:
"""
Loads a pretrained CNN model and presents a Gradio app to the user.
The user can upload a chest X-Ray image which will be preprocessed similarly
to the trained model, then run through the model.
A prediction of 1 or more of 14 diagnoses or no diagnosis will be output.
This will be followed up with suggested treatment from an OpenAI API.
"""

# Standard imports
import pandas as pd
from pathlib import Path
import numpy as np
import requests
from PIL import Image
import tensorflow as tf
from pathlib import Path
import gradio as gr
from gtts import gTTS
import random

# Gradio App Based on Aleksandar's Model

In [2]:
# first lets load our pre-trained model
xray_model = tf.keras.models.load_model(Path("Resources/detection_model.keras"))
xray_model

d:\Development\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


<Functional name=functional_1, built=True>

In [3]:
# Aleksandar's preprocessing of an image (not really used just a reference)
# Function to preprocess images (resize to desired dimensions)
def preprocess_image(img_path, target_size=(256, 256)):
    img = Image.open(img_path)
    img = img.resize(target_size)
    img = img.convert('RGB')  # Ensure all images are in RGB format
    img = np.array(img)
    return img / 255.0  # Normalize to range [0, 1]

In [4]:
# Load the CSV file that has list of all images, for random selection
df = pd.read_csv("Resources/Data_Entry_2017_v2020.csv")

In [5]:
# Define the load random function to load any of the images within the dataset (CSV)
def load_random(submit):
    idx = random.randrange(0, len(df["Image Index"]))
    img_name = df["Image Index"].iloc[idx]
    img_name = f"./Images/{img_name}"
    return img_name


In [6]:
def enable_submit(submit):
    return gr.update(interactive=True)

def disable_submit(submit):
    return gr.update(interactive=False)


In [7]:
# Define the function of our model that will process the image and meta data to make a prediction
def do_predict(img):
    # Process image based on preprocess code and prep to enter into model
    img = Image.fromarray(img)
    img = img.resize((256, 256))
    img = img.convert('RGB')  # Ensure all images are in RGB format
    img = np.array(img)
    img = img / 255.0 # Normalize to range [0, 1]
    img = np.array(img)
    img = np.expand_dims(img, axis=0)

    prediction = xray_model.predict(img)

    if prediction[0][0] < 0.5:
        output = f"This model found nothing in the X-ray. There is only a {prediction[0][0]*100:.0f}% chance of a diagnosis. Disclaimer: This app should only be used by doctors as a cross reference with standard radiology."
    else:
        output = f"This model suspects a diagnosis in this X-ray with a {prediction[0][0]*100:.0f}% certainty. Human analysis is required to determine the exact diagnosis."

    wavobj = gTTS(text=output, lang="en", slow=False)
    wavobj.save("output.wav")
 
    return output, "output.wav"

In [8]:
# Create an instance of the Gradio Blocks as a dynamic application

with gr.Blocks(theme="abidlabs/dracula_revamped") as app:
        title = gr.Markdown("# Chest X-Ray Cross Analysis")
        rand = gr.Button(value="Load Random Image from Dataset.")
        image = gr.Image(label="Or Upload Your Own.", value="PIL.Image.Image", interactive=True)
        submit = gr.Button(value="Classify Image", interactive=False)
        out_txt = gr.Textbox(lines=3, label="Text Prediction", show_copy_button=True)
        out_aud = gr.Audio(autoplay=True, label="Audio Prediction", interactive=False)
        image.change(enable_submit, inputs=submit, outputs=submit)
        image.clear(disable_submit, inputs=submit, outputs=submit)
        rand.click(load_random, inputs=rand, outputs=image)
        submit.click(do_predict, inputs=image, outputs=[out_txt, out_aud])
# Launch the app
app.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://eed47d9ec9867b816c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
